> **Note:** In most sessions you will be solving exercises posed in a Jupyter notebook that looks like this one. Because you are cloning a Github repository that only we can push to, you should **NEVER EDIT** any of the files you pull from Github. Instead, what you should do, is either make a new notebook and write your solutions in there, or **make a copy of this notebook and save it somewhere else** on your computer, not inside the `sds` folder that you cloned, so you can write your answers in there. If you edit the notebook you pulled from Github, those edits (possible your solutions to the exercises) may be overwritten and lost the next time you pull from Github. This is important, so don't hesitate to ask if it is unclear.

## Use Connector class for accessing the internet
Even if logging is not important for the below exercises, get in the habit of using this class for connecting to the internet, to practice logging your activity. This will be expected in the final exam.

You should run `pip install scraping_class` to install the module to be used.

In [3]:
import requests
from bs4 import BeautifulSoup
import re
import selenium
import time

In [4]:
import scraping_class
logfile = 'log_lisbeth.csv'## name your log file.
connector = scraping_class.Connector(logfile)


# Exercise Set 6: Introduction to Web Scraping

In this Exercise Set we shall practice our webscraping skills utiilizing only basic python.  
We shall cover variations between static and dynamic pages and build. 

## Exercise Section 6.1: Scraping Jobnet.dk

This exercise you get to practice locating the request that the JavaScript sends to get the job data that it builds the joblistings from. You should use the **>Network Monitor<** tool in your browser.

Furthermore you practice spotting how the pagination is done, without clicking on the next page button, but instead changing a small parameter in the URL.

> **Ex. 6.1.1:** Go to  www.jobnet.dk and investigate the page. Figure our waht url you need to scrape to collect jobposting data. Sometimes this can be hard and requires you to inspect the page and look in the **>Network Monitor<** by filtering by XHR files.

> **Ex. 6.1.2.:** Use the `request` module to collect the first 20 results and unpack the relevant `json` data into a `pandas` DataFrame.

> **Ex. 6.1.3.:** Store the 'TotalResultCount' value for later use.

In [78]:
import requests
import pandas as pd

In [79]:
url = 'https://job.jobnet.dk/CV/FindWork/Search'
response = requests.get(url)
jobs = response.json()
jobs = pd.DataFrame(response.json()['JobPositionPostings'])
TotalResultCount = response.json()['TotalResultCount']

> **Ex. 6.1.4:** This exercise is about paging the results. We need to understand the websites pagination scheme. 

> Now scroll down the webpage and press the next page button. See how the parameters of the url changes as you turn the pages.

> **Ex. 6.1.5:** Design a`for` loop using the `range` function that changes this paging parameter in the URL. Use the TotalResultCount parameter from before to define the limits of the range function. Store these urls in a container. 

>**extra** Change the SortValue parameter from BestMatch to CreationDate, to make the sorting amendable to updating results daily.

*(HINT: See that the parameter is an offset and that this relates to the number of results pr. call made.)*

In [97]:
TotalResultCount = int(TotalResultCount) # converting TotalResultCount to interger so that it can be used in range

def link_page(n): # defining a function that creates a link for a given page number
    link =  f'https://job.jobnet.dk/CV/FindWork/Search?Offset={n}&SortValue=BestMatch'
    return link


links = [link_page(n) for n in range(0,TotalResultCount,20)] #storing links for all the pages

> **Ex.6.1.6:** Pick 20 random links using the `random.sample()` function and collect them using the `Connector` class. Also use the `time.sleep()` function to limit the rate of your calls. Make sure to save the links already collected in a `set()` container to avoid having to reload links already collected. ***extra***: monitor the time left to completing the loop by using `tqdm.tqdm()` function.

> **Ex.6.1.7:** Load all the results into a DataFrame.

In [100]:
import random
import time
import tqdm
import requests

# draw random sample of links and store them in set

done = set(random.sample(links,20))

# create empty list

data = [] # create empty list

# collect links using the connector class

for url in tqdm.tqdm(done):
    response, id_ = connector.get(url, 'jobs') 
    if response.ok:
        d = response.json()
    else:
        print('Error')
    data += d['JobPositionPostings']
    time.sleep(0.5)
    
# loading results into a dataframe

df = pd.DataFrame(data)
df.sample(5)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:29<00:00,  1.50s/it]


,AutomatchType,Abroad,Weight,Title,JobHeadline,Presentation,HiringOrgName,WorkPlaceAddress,WorkPlacePostalCode,WorkPlaceCity,...,HiringOrgCVR,UserLoggedIn,AnonymousEmployer,ShareUrl,DetailsUrl,JobLogUrl,HasLocationValues,ID,Latitude,Longitude
194,0,False,1.0,Pladsansvarlig - reparatør,Pladsansvarlig - reparatør,,ALLROUND Byg A/S,Hans Egedes Vej 20,5210,Odense NV,...,27379869,False,False,https://job.jobnet.dk/CV/FindWork/DetailsSocia...,https://job.jobnet.dk/CV/FindWork/Details/5189407,https://job.jobnet.dk/CV/FindWork/Details/5189407,True,5189407,55.4178,10.3424
293,0,False,1.0,Faglig pædagogisk leder søges til Femkanten.,Faglig pædagogisk leder søges til Femkanten.,<p>Faglig pædagogisk leder søges til Femkanten...,Ishøj Kommune,Strandgården 17 B,2635,Ishøj,...,11931316,False,False,https://job.jobnet.dk/CV/FindWork/DetailsSocia...,https://job.jobnet.dk/CV/FindWork/Details/5174390,https://job.jobnet.dk/CV/FindWork/Details/5174390,True,5174390,55.5878,12.3948
389,0,False,1.0,tjener,tjener,,Highway 69 ApS,Frederikshavnsvej 69,9300,Sæby,...,40595643,False,False,https://job.jobnet.dk/CV/FindWork/DetailsSocia...,https://job.jobnet.dk/CV/FindWork/Details/5185661,https://job.jobnet.dk/CV/FindWork/Details/5185661,True,5185661,57.3439,10.5141
335,0,False,1.0,Sygeplejerske til stort akuthus med mange fagl...,Sygeplejerske til stort akuthus med mange fagl...,Du bliver en del af akuthuset på Psykiatrisk C...,Psykiatrisk Center Ballerup,Maglevænget 2,2750,Ballerup,...,29190623,False,False,https://job.jobnet.dk/CV/FindWork/DetailsSocia...,https://job.jobnet.dk/CV/FindWork/Details/5177384,https://job.jobnet.dk/CV/FindWork/Details/5177384,True,5177384,55.7207,12.3589
52,0,False,1.0,Teamleder,Norsktalende salgs- og kunderådgiver til Billi...,Er du norsktalende og brenner for godt salg og...,Brødrene A. & O. Johansen A/S,Rørvang 3,2620,Albertslund,...,0,False,False,https://candidate.hr-manager.net/ApplicationIn...,None,https://candidate.hr-manager.net/ApplicationIn...,True,E7673210,55.6656,12.3635


## Exercise Section 6.2: Scraping Trustpilot.com
Now for a slightly more elaborate, yet still simple scraping problem. Here we want to scrape trustpilot for user reviews. This data is very nice since it provides free labeled data (rating) to train a machine learning model to understand positive and negative sentiment. 

Here you will practice crawling a website collecting the links to each company review page, and finally locate another behind the scenes JavaScript request that gets the review data in a neat json format.

> **Ex. 6.2.1:** Visit the https://www.trustpilot.com/ website and locate the categories page.
From this page you find links to company listings.

> **Ex. 6.2.2:**
Get the category page using the `requests` module and extract each link to a specific category page from the HTML. This can be done using the basic python `.split()` string method. Make sure only links within the ***/categories/*** section are kept, checking each string using the ```if 'pattern' in string``` condition. 

*(Hint: The links are relative. You need to add the domain name)*


In [31]:
url_cat = 'https://www.trustpilot.com/categories'
links = set()

# fetching html markup behind categories page

response,call_id = connector.get(url_cat,'categories')
if response.ok:
    html = response.text
else:
    print('error')
    

link_list = html.split('href="')[1:] # split html text file into a list using "href=" as a delimitator

# take each link in link_list, split each link into a list using ' " ' as a delimitator.
# If the first element of the list contains the string '/categories/', then add that element to the set
# This loop returns a list of categories (x,y) in the format ["categories/x", "categories/y"]

for link_loc in link_list: 
    link = link_loc.split('"')[0] 
    if '/categories/' in link:  
        links.add(link) 

# Create a list of links that include the domain and category

links = ['https://www.trustpilot.com'+link for link in links]

> **Ex. 6.2.3:** Get one of the category section links. Write a function to extract the links to the company review page from the HTML.

> **Ex. 6.2.4:** Figure out how the pagination is done, by following how the url changes when pressing the **next page**-button to obtain more company listings. Write a function that builds links to paging all the company listing results of each category. This includes parsing the number of subpages of each category and changing the correct parameter in the url.

(Hint: Find the maximum number of result pages, right before the next page button and make a loop change the page parameter of the url.)


In [12]:
#[Answer to Ex.6.2.3-4]

> **Ex. 6.2.5:** Loop through all categories and build the paging links using the above defined function.

> **Ex. 6.2.6:** Randomly pick one of category listing links you have generated, and get the links to the companies listed using the other function defined. 

> **Ex. 6.2.7:** Visit one of these links and inspect the **>Network Monitor<** to locate the request that loads the review data. Use the requests module to retrieve this link and unpack the json results to a pandas DataFrame.


In [15]:
#[Answer to Ex.6.2.5-7]